In [1]:
import cv2
import numpy as numpy

from sklearn.metrics import pairwise

In [2]:
# Global variable

background = None
accumulated_weigh = 0.5

roi_top = 20
roi_bottom = 300
roi_right = 300
roi_left = 600

In [3]:
def calc_accum_avg(frame, accumulayed_weigh):
    global background
    if background is None:
        background = frame.copy().astype('float')
        return None

    cv2.accumulatedWeighted(frame, background, accumulated_weigh)

In [4]:
def segment(frame, threshold=25):
    global background
    diff = cv2.absdiff(background.astype('uint8'), frame)
    _, thresholded = cv2.threshold(diff, threshold, 255, cv2.THRESH_BINARY)
    
    contours, _ = cv2.findContours(thresholded.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if len(contours) == 0:
        return None
    else:
        hand_segment = max(contours, key=cv2.contourArea)
        return (thresholded, hand_segment)

In [ ]:
def count_fingers(thresholded, hand_segment):
    conv_hull = cv2.convexHull(hand_segment)

    top    = tuple(conv_hull[conv_hull[:, :, 1].argmin()][0])
    bottom = tuple(conv_hull[conv_hull[:, :, 1].argmax()][0])
    left   = tuple(conv_hull[conv_hull[:, :, 0].argmin()][0])
    right  = tuple(conv_hull[conv_hull[:, :, 0].argmax()][0])